In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
data =pd.read_csv('XAUUSD.csv')

# Splitting the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

NameError: name 'pd' is not defined

In [ ]:

# Normalize the input features using Min-Max scaler
scaler = MinMaxScaler()
train_data[features] = scaler.fit_transform(train_data[features])
test_data[features] = scaler.transform(test_data[features])

# Convert breakout direction labels to numerical values if necessary
train_data['BreakoutDirection'] = train_data['BreakoutDirection'].map({'no breakout': 0, 'bullish breakout': 1, 'bearish breakout': -1})
test_data['BreakoutDirection'] = test_data['BreakoutDirection'].map({'no breakout': 0, 'bullish breakout': 1, 'bearish breakout': -1})


In [ ]:

# Convert the data into sequences of fixed length
sequence_length = 10

def create_sequences(data):
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[features].values[i:i+sequence_length])
        y.append(data[target].values[i+sequence_length])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data)
X_test, y_test = create_sequences(test_data)

# Convert NumPy arrays to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()  # use long tensor for classification
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()


In [ ]:

# Step 3: Build the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

input_size = len(features)
hidden_size = 64
output_size = 3  # 3 classes: no breakout, bullish breakout, bearish breakout

model = LSTMModel(input_size, hidden_size, output_size)


In [ ]:

# Step 4: Training the Model

# Define hyperparameters
learning_rate = 0.001
batch_size = 32
num_epochs = 50

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

# Create DataLoader objects for training and testing
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:

# Training loop
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    # Print the loss after every epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

# Step 5: Evaluation and Prediction

# Evaluate the model on the testing set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted_labels = torch.max(test_outputs, 1)  # get predicted labels

# Calculate accuracy
accuracy = (predicted_labels == y_test).sum().item() / len(y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Make predictions on unseen data
unseen_data = ...  # unseen data to predict on
unseen_data[features] = scaler.transform(unseen_data[features])
X_unseen, _ = create_sequences(unseen_data)
X_unseen = torch.from_numpy(X_unseen).float()

model.eval()
with torch.no_grad():
    unseen_outputs = model(X_unseen)
    _, unseen_predicted_labels = torch.max(unseen_outputs, 1)  # get predicted labels

# inverse_transform the predicted labels if necessary
predicted_breakout_directions = unseen_predicted_labels.numpy()
